In [20]:
import pandas as pd

In [ ]:
# used modified parser from here: https://github.com/rathishg/drugbank_ca/blob/master/drug_bank_xml2db.py

In [17]:
# read, parse, process, and write drugbank files

!python3 drugbank_xml_2_db.py -i data/drugbank/full_database.xml

Reading and parsing xml file.
Processing records.
Writing output files.
Done:
	drug_target: 19028 records
	drug_target_action: 212473 records
	drugs: 14594 records
	targets: 19028 records


In [30]:
# convert txt files to pandas dfs

drug_target = pd.read_csv("data/drugbank/full_database.xml.drug_target.txt", sep='\t')
drug_target_action = pd.read_csv("data/drugbank/full_database.xml.drug_target_action.txt", sep="\t")
drugs = pd.read_csv("data/drugbank/full_database.xml.drugs.txt", sep="\t")
targets = pd.read_csv("data/drugbank/full_database.xml.targets.txt", sep='\t')


In [90]:
# format data by lower casing all drug names

drugs['drug_name'] = [d.lower() for d in drugs['drug_name']]

heart_drugs = pd.read_csv("data/drugs_with_heart_se.csv")
heart_drugs['drug_name'] = [d.lower() for d in heart_drugs['drug_name']]

In [72]:
# create db of all drugs that give heart-related side effects and their targets

heart_drugs_db = drugs[drugs['drug_name'].isin(list(heart_drugs.drug_name))]

In [96]:
# num unique drugs that had target info

print("num unique drugs with target info: " + str(len(heart_drugs_db.drug_id.unique())))

num unique drugs with target info: 719


In [98]:
# num unique targets 

heart_drug_targets = drug_target_action[drug_target_action['drug_id'].isin(heart_drugs_db.drug_id)]
print("num unique targets: " + str(len(heart_drug_targets.target_id.unique())))

num unique targets: 790
